In [ ]:
# Input handling

with open("Day10_ex_input.txt", 'r') as input_file:
    input_raw = input_file.read().split('\n')

In [ ]:
# Split input using regex
import re

light_indicators_re = re.compile("\[.*?\]")
joltages_re = re.compile("\{.*?\}")
buttons_re = re.compile("\(.*?\)")


In [ ]:
light_indicators = {}
for machine, line in enumerate(input_raw):
    light_indicator = light_indicators_re.findall(line)[0][1:-1]
    light_indicator_bool = [light == '#' for light in light_indicator]
    light_indicators[machine] = light_indicator_bool

In [ ]:
joltages = {}
for machine, line in enumerate(input_raw):
    joltage = joltages_re.findall(line)[0][1:-1]

    joltages[machine] = [int(number) for number in joltage.split(',')]

In [ ]:
all_buttons = {}
for machine, line in enumerate(input_raw):
    machine_buttons_iterator = buttons_re.finditer(line)
    machine_buttons_raw = [button.group()[1:-1] for button in machine_buttons_iterator]
    machine_buttons = [tuple(map(int, button.split(','))) for button in machine_buttons_raw]

    machine_buttons_bool = []

    for machine_button in machine_buttons:
        machine_button_bool = [light in machine_button for light in range(len(light_indicators[machine]))]
        machine_buttons_bool.append(machine_button_bool)

    all_buttons[machine] = machine_buttons_bool

In [ ]:
# Try machine 1
m1_lights = light_indicators[0]
m1_buttons = all_buttons[0]

print(m1_lights)
print(m1_buttons)

In [ ]:
# press button 1
pressed_button = 1
m1_lights_altered = [m1_lights[i] ^ m1_buttons[pressed_button][i] for i in range(len(m1_lights))]
m1_lights_altered